In [30]:
#CSCE 5214 Random Selection Deep Learning
#this notebook is partly based on information from this link https://www.kaggle.com/code/virajkadam/birdclef-bird-sound-classification#4.-Load-training-samples
!pip install noisereduce

In [31]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [32]:
import noisereduce as nr
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator

In [35]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_directory = '/content/drive/MyDrive/P2_DeepLearning/birdclef-2023/random10_prep_data/train' #paths to directories with the spectrographs
validation_directory = '/content/drive/MyDrive/P2_DeepLearning/birdclef-2023/random10_prep_data/test'

#train_directory = r'/content/drive/MyDrive/P2_DeepLearning/birdclef-2023.zip (Unzipped Files)/random10_prep_data/train'
#validation_directory = r'/content/drive/MyDrive/P2_DeepLearning/birdclef-2023.zip (Unzipped Files)/random10_prep_data/test'

#reserved_directory = '/content/drive/MyDrive/P2_DeepLearning/birdclef-2023.zip (Unzipped Files)/random10_prep_data/reserved'

batchSize = 8

# Image dimensions
img_width, img_height = 322, 884
train_datagen = ImageDataGenerator(rescale=1./255) #divides for scaling
validation_datagen = ImageDataGenerator(rescale=1./255)
reserved_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory( #loads images for training/validation
    #train_directory,
    directory = train_directory,
    target_size=(img_height, img_width),
    color_mode = "grayscale",
    batch_size = batchSize,
    class_mode='categorical',
    shuffle = True,
    seed = 91825)

validation_generator = validation_datagen.flow_from_directory(
    #validation_directory,
    directory = validation_directory,
    target_size=(img_height, img_width),
    color_mode = "grayscale",
    batch_size = batchSize,
    class_mode='categorical',
    shuffle = True,
    seed = 18232)


Found 1836 images belonging to 10 classes.
Found 572 images belonging to 10 classes.


In [36]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow import keras
# Define the CNN model
model = Sequential()

# Convolutional layer with 32 filters, a kernel size of 3x3, and ReLU activation
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(322, 884, 1) ) )
model.add(MaxPooling2D((2, 2)))

# Adding another convolutional layer
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((5, 5)))

#model.add(Conv2D(64, (5, 5), activation='relu'))
#model.add(MaxPooling2D((4, 4)))

#Adding dropout layer to control overfitting
model.add(Dropout(0.25))
# Adding dense softmax layer
model.add(Dense(128, activation='softmax'))

# Adding layer with tanh acitvation function to improve accuracy
model.add(Conv2D(128, (3, 3), activation='tanh'))
#varying the size of the max pooling kernel has helepd the model
model.add(MaxPooling2D((4, 4)))

# Flatten the output of the convolutional layers
model.add(Flatten())

# Fully connected layer with 128 units and ReLU activation
model.add(Dense(128, activation='relu'))

#several layers added


# Dropout for regularization
model.add(Dropout(0.15))

# Output layer with a unit for each class and softmax activation
model.add(Dense(10, activation='softmax'))

# Compile the model

#opti = keras.optimizers.Adam(learning_rate = 0.02)
model.compile( optimizer = 'adam',#optimizer = opti,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Model summary
model.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 320, 882, 32)      320       
                                                                 
 max_pooling2d_9 (MaxPoolin  (None, 160, 441, 32)      0         
 g2D)                                                            
                                                                 
 conv2d_10 (Conv2D)          (None, 158, 439, 64)      18496     
                                                                 
 max_pooling2d_10 (MaxPooli  (None, 31, 87, 64)        0         
 ng2D)                                                           
                                                                 
 dropout_6 (Dropout)         (None, 31, 87, 64)        0         
                                                                 
 dense_9 (Dense)             (None, 31, 87, 128)      

In [37]:
# Train the model

fitModel = model.fit(
    train_generator,
    steps_per_epoch = train_generator.samples // batchSize,
    epochs = 25,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batchSize)


Epoch 1/25
229/229 [==============================] - 35s 146ms/step - loss: 1.5130 - accuracy: 0.6171 - val_loss: 1.3308 - val_accuracy: 0.5915
Epoch 2/25
229/229 [==============================] - 34s 147ms/step - loss: 1.0061 - accuracy: 0.7002 - val_loss: 1.0927 - val_accuracy: 0.6426
Epoch 3/25
229/229 [==============================] - 29s 127ms/step - loss: 0.7681 - accuracy: 0.7664 - val_loss: 1.2824 - val_accuracy: 0.6831
Epoch 4/25
229/229 [==============================] - 30s 132ms/step - loss: 0.5819 - accuracy: 0.8085 - val_loss: 1.5109 - val_accuracy: 0.6778
Epoch 5/25
229/229 [==============================] - 33s 142ms/step - loss: 0.4279 - accuracy: 0.8523 - val_loss: 1.3169 - val_accuracy: 0.6866
Epoch 6/25
229/229 [==============================] - 29s 125ms/step - loss: 0.2955 - accuracy: 0.8944 - val_loss: 1.3857 - val_accuracy: 0.6972
Epoch 7/25
229/229 [==============================] - 30s 132ms/step - loss: 0.2218 - accuracy: 0.9245 - val_loss: 1.6197 - val_ac

In [38]:
test_directory = '/content/drive/MyDrive/P2_DeepLearning/birdclef-2023/random10_prep_data/test'

test_datagen = ImageDataGenerator(rescale=1./255, validation_split = 0.9625)

test_generator = test_datagen.flow_from_directory(
    directory = test_directory,
    target_size=(img_height, img_width),
    color_mode = "grayscale",
    batch_size= batchSize,
    class_mode='categorical',
    subset = 'validation',
    shuffle = False) #since it's test, don't set shuffle to true


Found 544 images belonging to 10 classes.


In [39]:
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // batchSize)
print("Test accuracy:", test_accuracy)
print("Test loss:", test_loss)


68/68 [==============================] - 6s 86ms/step - loss: 3.3352 - accuracy: 0.7169
Test accuracy: 0.716911792755127
Test loss: 3.335244655609131


In [40]:
from sklearn.metrics import classification_report, confusion_matrix
import random
# # Predict classes
predictions = model.predict(test_generator, steps=test_generator.samples // batchSize)
predicted_classes = np.argmax(predictions, axis=1)

# # Get true labels
#true_classes = random.sample(test_generator.classes, 544)
true_classes = test_generator.classes
#scores report
report = classification_report(true_classes, predicted_classes, target_names=test_generator.class_indices.keys(), zero_division = 0)

print(report)


68/68 [==============================] - 6s 79ms/step
              precision    recall  f1-score   support

     afbfly1       0.20      0.14      0.17        28
     afdfly1       0.69      0.81      0.74        36
     chespa1       0.20      1.00      0.33         1
     chewea1       0.80      0.44      0.57         9
     grywrw1       0.90      0.78      0.84        49
     luebus1       0.69      0.65      0.67        31
     nobfly1       0.54      0.20      0.29        35
     rocmar2       0.21      0.31      0.25        26
     somgre1       0.83      0.88      0.86       304
     somtit4       0.39      0.44      0.42        25

    accuracy                           0.72       544
   macro avg       0.55      0.56      0.51       544
weighted avg       0.72      0.72      0.71       544



In [41]:
model.save('/content/drive/MyDrive/P2_DeepLearning/birdclef-2023/prep_data/RandomSelection_BirdPred.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
